# Combined Sensitivity Analysis

12-panel figure comparing YLL sensitivity (left column), GHG price sensitivity (middle column), and combined GHG+YLL sensitivity (right column).

- Row 1: Food consumption (kcal/person/day)
- Row 2: GHG emissions by food group (GtCO2eq)
- Row 3: Health cost by food group (million YLL)
- Row 4: Objective breakdown (billion USD)

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from sensitivity_utils import (
    PRETTY_NAMES_HEALTH,
    aggregate_food_groups,
    assign_food_colors,
    extract_combined_param_value,
    extract_consumption_data,
    extract_ghg_data,
    extract_health_data,
    extract_objective_data,
    extract_param_value,
    load_food_to_group,
    plot_objective_sensitivity,
    plot_stacked_sensitivity,
    prepare_objective_data,
    set_dual_xaxis_labels,
    set_dual_xlabel,
)

In [ ]:
# Configuration
import yaml

PROJECT_ROOT = Path("..").resolve()
CACHE_DIR = Path("cache")  # Relative to notebooks/

# YLL config
YLL_CONFIG_NAME = "yll"
YLL_RESULTS_DIR = PROJECT_ROOT / "results" / YLL_CONFIG_NAME
YLL_PROCESSING_DIR = PROJECT_ROOT / "processing" / YLL_CONFIG_NAME
YLL_CACHE_DIR = CACHE_DIR / YLL_CONFIG_NAME

# GHG config
GHG_CONFIG_NAME = "ghg"
GHG_RESULTS_DIR = PROJECT_ROOT / "results" / GHG_CONFIG_NAME
GHG_PROCESSING_DIR = PROJECT_ROOT / "processing" / GHG_CONFIG_NAME
GHG_CACHE_DIR = CACHE_DIR / GHG_CONFIG_NAME

# Combined GHG+YLL config
GHG_YLL_CONFIG_NAME = "ghg_yll"
GHG_YLL_RESULTS_DIR = PROJECT_ROOT / "results" / GHG_YLL_CONFIG_NAME
GHG_YLL_PROCESSING_DIR = PROJECT_ROOT / "processing" / GHG_YLL_CONFIG_NAME
GHG_YLL_CACHE_DIR = CACHE_DIR / GHG_YLL_CONFIG_NAME

# Load food to group mapping
FOOD_TO_GROUP = load_food_to_group(PROJECT_ROOT)

# Constants
CONSTANT_HEALTH_VALUE_PER_YLL = 10000
CONSTANT_GHG_PRICE = 100
N_WORKERS = 8


def load_defined_scenarios(config_name: str) -> set[str]:
    """Load scenario names defined in the scenarios YAML file."""
    # First check if config references a separate scenarios file
    config_path = PROJECT_ROOT / "config" / f"{config_name}.yaml"
    with open(config_path) as f:
        config = yaml.safe_load(f)

    scenarios_file = config.get("scenario_defs")
    if scenarios_file:
        scenarios_path = PROJECT_ROOT / scenarios_file
    else:
        scenarios_path = PROJECT_ROOT / "config" / f"{config_name}_scenarios.yaml"

    with open(scenarios_path) as f:
        scenarios = yaml.safe_load(f)

    return set(scenarios.keys())


YLL_DEFINED_SCENARIOS = load_defined_scenarios(YLL_CONFIG_NAME)
GHG_DEFINED_SCENARIOS = load_defined_scenarios(GHG_CONFIG_NAME)
GHG_YLL_DEFINED_SCENARIOS = load_defined_scenarios(GHG_YLL_CONFIG_NAME)

print(f"YLL defined scenarios: {len(YLL_DEFINED_SCENARIOS)}")
print(f"GHG defined scenarios: {len(GHG_DEFINED_SCENARIOS)}")
print(f"GHG_YLL defined scenarios: {len(GHG_YLL_DEFINED_SCENARIOS)}")

## Load YLL Data

In [ ]:
# Find all YLL scenarios (filtered by defined scenarios)
yll_solved_dir = YLL_RESULTS_DIR / "solved"
yll_network_files = list(yll_solved_dir.glob("model_scen-yll_*.nc"))

yll_scenarios = []
for f in yll_network_files:
    scenario = f.stem.replace("model_scen-", "")
    # Only include if defined in scenarios YAML
    if scenario not in YLL_DEFINED_SCENARIOS:
        continue
    yll_value = extract_param_value(scenario, "yll")
    if yll_value is not None:
        yll_scenarios.append((yll_value, scenario, f))

yll_scenarios.sort(key=lambda x: x[0])
print(f"Found {len(yll_scenarios)} YLL scenarios")

In [ ]:
# Extract YLL consumption data
df_yll_consumption = extract_consumption_data(
    yll_scenarios,
    FOOD_TO_GROUP,
    YLL_CACHE_DIR / "consumption.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

# Aggregate and prepare for plotting
df_yll_consumption_plot = aggregate_food_groups(df_yll_consumption)
min_yll = df_yll_consumption_plot.index.min()
yll_group_order = (
    df_yll_consumption_plot.loc[min_yll].sort_values(ascending=False).index.tolist()
)
df_yll_consumption_plot = df_yll_consumption_plot[yll_group_order]
yll_colors = assign_food_colors(df_yll_consumption_plot)

print(f"YLL consumption data shape: {df_yll_consumption_plot.shape}")

In [ ]:
# Extract YLL objective data
df_yll_obj = extract_objective_data(
    yll_scenarios,
    YLL_CACHE_DIR / "objective_breakdown.csv",
    param_name="yll_value",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)
df_yll_obj = prepare_objective_data(df_yll_obj)
print(f"YLL objective data shape: {df_yll_obj.shape}")

In [ ]:
# Extract YLL GHG emissions data
df_yll_ghg = extract_ghg_data(
    yll_scenarios,
    FOOD_TO_GROUP,
    YLL_CACHE_DIR / "ghg_by_food_group.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as consumption plot
df_yll_ghg_plot = aggregate_food_groups(df_yll_ghg)
available_groups = [g for g in yll_group_order if g in df_yll_ghg_plot.columns]
df_yll_ghg_plot = df_yll_ghg_plot[available_groups]

print(f"YLL GHG data shape: {df_yll_ghg_plot.shape}")

In [ ]:
# Extract YLL health cost data
df_yll_health = extract_health_data(
    yll_scenarios,
    YLL_PROCESSING_DIR,
    YLL_CACHE_DIR / "health_by_food_group.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

# Aggregate fruits+vegetables to match other panels
df_yll_health_plot = aggregate_food_groups(df_yll_health)

# Use available groups that match the health risk factors
yll_health_groups = [g for g in yll_group_order if g in df_yll_health_plot.columns]
df_yll_health_plot = df_yll_health_plot[yll_health_groups]

print(f"YLL health data shape: {df_yll_health_plot.shape}")

## Load GHG Data

In [ ]:
# Find all GHG scenarios (filtered by defined scenarios)
ghg_solved_dir = GHG_RESULTS_DIR / "solved"
ghg_network_files = list(ghg_solved_dir.glob("model_scen-ghg_*.nc"))

ghg_scenarios = []
for f in ghg_network_files:
    scenario = f.stem.replace("model_scen-", "")
    # Only include if defined in scenarios YAML
    if scenario not in GHG_DEFINED_SCENARIOS:
        continue
    ghg_price = extract_param_value(scenario, "ghg")
    if ghg_price is not None:
        ghg_scenarios.append((ghg_price, scenario, f))

ghg_scenarios.sort(key=lambda x: x[0])
print(f"Found {len(ghg_scenarios)} GHG scenarios")

In [ ]:
# Extract GHG consumption data
df_ghg_consumption = extract_consumption_data(
    ghg_scenarios,
    FOOD_TO_GROUP,
    GHG_CACHE_DIR / "consumption.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as YLL plots (for consistent colors across columns)
df_ghg_consumption_plot = aggregate_food_groups(df_ghg_consumption)
available_groups_ghg = [
    g for g in yll_group_order if g in df_ghg_consumption_plot.columns
]
df_ghg_consumption_plot = df_ghg_consumption_plot[available_groups_ghg]

print(f"GHG consumption data shape: {df_ghg_consumption_plot.shape}")

In [ ]:
# Extract GHG objective data
df_ghg_obj = extract_objective_data(
    ghg_scenarios,
    GHG_CACHE_DIR / "objective_breakdown.csv",
    param_name="ghg_price",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)
df_ghg_obj = prepare_objective_data(df_ghg_obj)
print(f"GHG objective data shape: {df_ghg_obj.shape}")

In [ ]:
# Extract GHG GHG emissions data
df_ghg_ghg = extract_ghg_data(
    ghg_scenarios,
    FOOD_TO_GROUP,
    GHG_CACHE_DIR / "ghg_by_food_group.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as YLL plots (for consistent colors across columns)
df_ghg_ghg_plot = aggregate_food_groups(df_ghg_ghg)
available_groups_ghg_ghg = [g for g in yll_group_order if g in df_ghg_ghg_plot.columns]
df_ghg_ghg_plot = df_ghg_ghg_plot[available_groups_ghg_ghg]

print(f"GHG GHG data shape: {df_ghg_ghg_plot.shape}")

In [ ]:
# Extract GHG health cost data
df_ghg_health = extract_health_data(
    ghg_scenarios,
    GHG_PROCESSING_DIR,
    GHG_CACHE_DIR / "health_by_food_group.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate fruits+vegetables to match other panels
df_ghg_health_plot = aggregate_food_groups(df_ghg_health)

# Use available groups that match YLL health data (for consistent colors)
ghg_health_groups = [g for g in yll_health_groups if g in df_ghg_health_plot.columns]
df_ghg_health_plot = df_ghg_health_plot[ghg_health_groups]

print(f"GHG health data shape: {df_ghg_health_plot.shape}")

## Load GHG+YLL Data (Combined)

In [ ]:
# Find all GHG_YLL scenarios (filtered by defined scenarios)
ghg_yll_solved_dir = GHG_YLL_RESULTS_DIR / "solved"
ghg_yll_network_files = list(ghg_yll_solved_dir.glob("model_scen-ghg_yll_*.nc"))

ghg_yll_scenarios = []
for f in ghg_yll_network_files:
    scenario = f.stem.replace("model_scen-", "")
    # Only include if defined in scenarios YAML
    if scenario not in GHG_YLL_DEFINED_SCENARIOS:
        continue
    params = extract_combined_param_value(scenario)
    if params is not None:
        ghg_price, yll_value = params
        # Use ghg_price as the primary index (yll_value = ghg_price * 100)
        ghg_yll_scenarios.append((ghg_price, scenario, f))

ghg_yll_scenarios.sort(key=lambda x: x[0])
print(f"Found {len(ghg_yll_scenarios)} GHG_YLL scenarios")
print(f"GHG prices: {[s[0] for s in ghg_yll_scenarios]}")

In [ ]:
# Extract GHG_YLL consumption data
df_ghg_yll_consumption = extract_consumption_data(
    ghg_yll_scenarios,
    FOOD_TO_GROUP,
    GHG_YLL_CACHE_DIR / "consumption.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as YLL plots (for consistent colors)
df_ghg_yll_consumption_plot = aggregate_food_groups(df_ghg_yll_consumption)
available_groups_ghg_yll = [
    g for g in yll_group_order if g in df_ghg_yll_consumption_plot.columns
]
df_ghg_yll_consumption_plot = df_ghg_yll_consumption_plot[available_groups_ghg_yll]

print(f"GHG_YLL consumption data shape: {df_ghg_yll_consumption_plot.shape}")

In [ ]:
# Extract GHG_YLL objective data
# For combined sensitivity, use actual values from scenarios (varying with ghg_price)
df_ghg_yll_obj = extract_objective_data(
    ghg_yll_scenarios,
    GHG_YLL_CACHE_DIR / "objective_breakdown.csv",
    param_name="ghg_price",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)
df_ghg_yll_obj = prepare_objective_data(df_ghg_yll_obj)
print(f"GHG_YLL objective data shape: {df_ghg_yll_obj.shape}")

In [ ]:
# Extract GHG_YLL GHG emissions data
df_ghg_yll_ghg = extract_ghg_data(
    ghg_yll_scenarios,
    FOOD_TO_GROUP,
    GHG_YLL_CACHE_DIR / "ghg_by_food_group.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as YLL plots
df_ghg_yll_ghg_plot = aggregate_food_groups(df_ghg_yll_ghg)
available_groups_ghg_yll_ghg = [
    g for g in yll_group_order if g in df_ghg_yll_ghg_plot.columns
]
df_ghg_yll_ghg_plot = df_ghg_yll_ghg_plot[available_groups_ghg_yll_ghg]

print(f"GHG_YLL GHG data shape: {df_ghg_yll_ghg_plot.shape}")

In [ ]:
# Extract GHG_YLL health cost data
df_ghg_yll_health = extract_health_data(
    ghg_yll_scenarios,
    GHG_YLL_PROCESSING_DIR,
    GHG_YLL_CACHE_DIR / "health_by_food_group.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate fruits+vegetables to match other panels
df_ghg_yll_health_plot = aggregate_food_groups(df_ghg_yll_health)

# Use available groups that match YLL health data (for consistent colors)
ghg_yll_health_groups = [
    g for g in yll_health_groups if g in df_ghg_yll_health_plot.columns
]
df_ghg_yll_health_plot = df_ghg_yll_health_plot[ghg_yll_health_groups]

print(f"GHG_YLL health data shape: {df_ghg_yll_health_plot.shape}")

## Combined 12-Panel Figure

In [ ]:
# X-axis configuration
YLL_XTICKS = [1, 10, 100, 1000, 10000, 100000]
YLL_XTICKLABELS = ["0", "10", "100", "1k", "10k", "100k"]
YLL_XLABEL = "Value per Year of Life Lost [USD/YLL]"

GHG_XTICKS = [1, 10, 100, 500]
GHG_XTICKLABELS = ["0", "10", "100", "500"]
GHG_XLABEL = "GHG price [USD/tCO2eq]"

# Combined GHG+YLL x-axis (both parameters vary together)
GHG_YLL_XTICKS = [1, 10, 100, 500]  # GHG price values
GHG_YLL_GHG_VALUES = [0, 10, 100, 500]  # Displayed GHG prices
GHG_YLL_YLL_VALUES = [0, 4000, 40000, 200000]  # Corresponding YLL values (= ghg * 400)

# Manual label positions for YLL plots
YLL_CONSUMPTION_LABEL_X = {
    "grain": 7,
    "dairy": 30,
    "starchy_vegetable": 5,
    "legumes": 3000,
    "oil": 5,
    "red_meat": 3,
    "sugar": 10,
    "nuts_seeds": 10000,
    "whole_grains": 1000,
    "fruits_vegetables": 300,
    "eggs_poultry": 30,
}

YLL_GHG_LABEL_X = {
    "red_meat": 3,
    "dairy": 30,
    "grain": 7,
    "oil": 10,
    "starchy_vegetable": 5,
    "legumes": 3000,
    "sugar": 5,
    "nuts_seeds": 30000,
    "whole_grains": 500,
    "fruits_vegetables": 3000,
    "eggs_poultry": 30,
}

YLL_OBJ_LABEL_X = {
    "Crop production": 50000,
    "Health burden": 10,
    "GHG cost": 3,
    "Trade": 100000,
    "Consumer values": 5000,
}

# Manual label positions for GHG plots
GHG_CONSUMPTION_LABEL_X = {
    "eggs_poultry": 8,
}

GHG_GHG_LABEL_SKIP = {"legumes", "fruits_vegetables"}

GHG_HEALTH_LABEL_SKIP = {"legumes"}

# GHG_YLL plots - skip small groups
GHG_YLL_LABEL_SKIP = {"legumes"}

In [ ]:
# Configuration option
INCLUDE_OBJECTIVE_ROW = True  # Set to False to hide the objective breakdown row

# Create multipanel figure with shared axes within rows
n_rows = 4 if INCLUDE_OBJECTIVE_ROW else 3
fig_height = 9.5 if INCLUDE_OBJECTIVE_ROW else 7.5
fig, axes = plt.subplots(n_rows, 3, figsize=(10.5, fig_height))

# Row 1: Food consumption
# Panel a: YLL food consumption (top-left)
plot_stacked_sensitivity(
    df_yll_consumption_plot,
    yll_colors,
    axes[0, 0],
    xlabel=YLL_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="a",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=YLL_CONSUMPTION_LABEL_X,
    y_max=2400,
)

# Panel b: GHG food consumption (top-middle)
plot_stacked_sensitivity(
    df_ghg_consumption_plot,
    yll_colors,
    axes[0, 1],
    xlabel=GHG_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="b",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_x_positions=GHG_CONSUMPTION_LABEL_X,
    y_max=2400,
)

# Panel c: GHG_YLL food consumption (top-right)
plot_stacked_sensitivity(
    df_ghg_yll_consumption_plot,
    yll_colors,
    axes[0, 2],
    xlabel="",  # Will be set with dual labels
    ylabel="Food consumption [kcal/person/day]",
    panel_label="c",
    x_ticks=GHG_YLL_XTICKS,
    x_ticklabels=[""] * len(GHG_YLL_XTICKS),  # Will be replaced
    label_skip=GHG_YLL_LABEL_SKIP,
    y_max=2400,
)

# Row 2: GHG emissions
# Panel d: YLL GHG emissions
plot_stacked_sensitivity(
    df_yll_ghg_plot,
    yll_colors,
    axes[1, 0],
    xlabel=YLL_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="d",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=YLL_GHG_LABEL_X,
    min_height_for_label=0.08,
)

# Panel e: GHG GHG emissions
plot_stacked_sensitivity(
    df_ghg_ghg_plot,
    yll_colors,
    axes[1, 1],
    xlabel=GHG_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="e",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_skip=GHG_GHG_LABEL_SKIP,
    min_height_for_label=0.08,
)

# Panel f: GHG_YLL GHG emissions
plot_stacked_sensitivity(
    df_ghg_yll_ghg_plot,
    yll_colors,
    axes[1, 2],
    xlabel="",
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="f",
    x_ticks=GHG_YLL_XTICKS,
    x_ticklabels=[""] * len(GHG_YLL_XTICKS),
    label_skip=GHG_YLL_LABEL_SKIP,
    min_height_for_label=0.08,
)

# Row 3: Health cost by food group
# Panel g: YLL health cost
plot_stacked_sensitivity(
    df_yll_health_plot,
    yll_colors,
    axes[2, 0],
    xlabel=YLL_XLABEL,
    ylabel="Health cost [million YLL]",
    panel_label="g",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    min_height_for_label=0.08,
    pretty_names=PRETTY_NAMES_HEALTH,
)

# Panel h: GHG health cost
plot_stacked_sensitivity(
    df_ghg_health_plot,
    yll_colors,
    axes[2, 1],
    xlabel=GHG_XLABEL,
    ylabel="Health cost [million YLL]",
    panel_label="h",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_skip=GHG_HEALTH_LABEL_SKIP,
    min_height_for_label=0.08,
    pretty_names=PRETTY_NAMES_HEALTH,
)

# Panel i: GHG_YLL health cost
plot_stacked_sensitivity(
    df_ghg_yll_health_plot,
    yll_colors,
    axes[2, 2],
    xlabel="",
    ylabel="Health cost [million YLL]",
    panel_label="i",
    x_ticks=GHG_YLL_XTICKS,
    x_ticklabels=[""] * len(GHG_YLL_XTICKS),
    label_skip=GHG_YLL_LABEL_SKIP,
    min_height_for_label=0.08,
    pretty_names=PRETTY_NAMES_HEALTH,
)

# Row 4: Objective breakdown (optional)
if INCLUDE_OBJECTIVE_ROW:
    # Panel j: YLL objective breakdown
    plot_objective_sensitivity(
        df_yll_obj,
        axes[3, 0],
        xlabel=YLL_XLABEL,
        panel_label="j",
        x_ticks=YLL_XTICKS,
        x_ticklabels=YLL_XTICKLABELS,
        health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
        ghg_price=CONSTANT_GHG_PRICE,
        label_x_positions=YLL_OBJ_LABEL_X,
        highlight_cat="GHG cost",
    )

    # Panel k: GHG objective breakdown
    plot_objective_sensitivity(
        df_ghg_obj,
        axes[3, 1],
        xlabel=GHG_XLABEL,
        panel_label="k",
        x_ticks=GHG_XTICKS,
        x_ticklabels=GHG_XTICKLABELS,
        health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
        ghg_price=CONSTANT_GHG_PRICE,
        highlight_cat="Health burden",
    )

    # Panel l: GHG_YLL objective breakdown
    plot_objective_sensitivity(
        df_ghg_yll_obj,
        axes[3, 2],
        xlabel="",
        panel_label="l",
        x_ticks=GHG_YLL_XTICKS,
        x_ticklabels=[""] * len(GHG_YLL_XTICKS),
        health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
        ghg_price=CONSTANT_GHG_PRICE,
    )

# Add column titles
axes[0, 0].set_title("Health value sensitivity", fontsize=9, fontweight="bold", pad=10)
axes[0, 1].set_title("GHG price sensitivity", fontsize=9, fontweight="bold", pad=10)
axes[0, 2].set_title("Combined sensitivity", fontsize=9, fontweight="bold", pad=10)

# Share y-axis limits within each row
for row in range(n_rows):
    y_min = min(ax.get_ylim()[0] for ax in axes[row, :])
    y_max = max(ax.get_ylim()[1] for ax in axes[row, :])
    for col in range(3):
        axes[row, col].set_ylim(y_min, y_max)

# Remove x-axis labels except for bottom row
for row in range(n_rows - 1):
    for col in range(3):
        axes[row, col].set_xlabel("")
        axes[row, col].set_xticklabels([])

# Set dual-colored x-axis labels for right column (bottom row only)
set_dual_xaxis_labels(
    axes[n_rows - 1, 2],
    GHG_YLL_XTICKS,
    GHG_YLL_GHG_VALUES,
    GHG_YLL_YLL_VALUES,
)
set_dual_xlabel(axes[n_rows - 1, 2])

# Remove y-axis labels from middle and right columns
for row in range(n_rows):
    for col in [1, 2]:
        axes[row, col].set_ylabel("")
        axes[row, col].set_yticklabels([])

# Align y-axis labels horizontally across rows
fig.align_ylabels(axes[:, 0])

plt.tight_layout()

# Add extra bottom margin for dual x-axis label on right column
plt.subplots_adjust(bottom=0.08)

# Save to notebooks/figures/
output_dir = PROJECT_ROOT / "notebooks" / "figures"
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / "combined_sensitivity.pdf"
plt.savefig(output_path, dpi=300, bbox_inches="tight")
print(f"Saved to: {output_path}")

plt.show()